In [1]:
import io
import urllib

import pandas as pd
import requests
from config import Config
from requests_ntlm import HttpNtlmAuth

In [2]:
class ContractMonitor:
    def __init__(self, ssrs_url, ssrs_usr, ssrs_pwd):
        self.ssrs_url = ssrs_url
        self.ssrs_usr = ssrs_usr
        self.ssrs_pwd = ssrs_pwd

    def set_params(self):
        self.payload = [
            ("Seas:isnull", True),
            ("Season", "ALL 22-23"),
            ("Season", "ALL 23-24"),
            ("City", "ALL"),
            ("Hotel_Chain", "ALL"),
            ("hotel", 1000000),
            ("operator", 2),
            ("CreateDateFrom:isnull", True),
            ("CreateDateTo:isnull", True),
            ("contracttype", "A"),
            # ("contracttype", "S"),
            ("contractstatus", "E"),
            # ('contractstatus', 'H'),
            ("Revize", 2),
            ("showauth", True),
            ("view", 1),
            ("citycontract", 1),
            ("citycontract", 0),
            ("rs:ParameterLanguage", ""),
            ("rs:Command", "Render"),
            ("rs:Format", "CSV"),
            ("rc:ItemPath", "table1"),
        ]

        self.params = urllib.parse.urlencode(self.payload, quote_via=urllib.parse.quote)

    def get(self):
        response = requests.get(
            self.ssrs_url,
            params=self.params,
            stream=True,
            auth=HttpNtlmAuth(self.ssrs_usr, self.ssrs_pwd),
        )

        print(response.url, response.status_code)
        if response.status_code == 200:
            data = response.content.decode("utf8")
            if len(data) > 424:
                return data
        return None

In [3]:
ssrs_url = (
    "http://95.211.42.206/ReportServer_SSRS?/DU Reports/Contracting/Contract Monitor"
)
ssrs_usr = Config.SSRS_USERNAME
ssrs_pwd = Config.SSRS_PASSWORD

contracts = ContractMonitor(ssrs_url, ssrs_usr, ssrs_pwd)
contracts.set_params()
data = contracts.get()

http://95.211.42.206/ReportServer_SSRS?/DU%20Reports/Contracting/Contract%20Monitor&Seas%3Aisnull=True&Season=ALL%2022-23&Season=ALL%2023-24&City=ALL&Hotel_Chain=ALL&hotel=1000000&operator=2&CreateDateFrom%3Aisnull=True&CreateDateTo%3Aisnull=True&contracttype=A&contractstatus=E&Revize=2&showauth=True&view=1&citycontract=1&citycontract=0&rs%3AParameterLanguage=&rs%3ACommand=Render&rs%3AFormat=CSV&rc%3AItemPath=table1 200


In [4]:
contracts = pd.read_csv(
    io.StringIO(data),
    header=0,
    skiprows=2,
    names=[
        "contract_id",
        "contract_type",
        "status",
        "purchase_contract_id",
        "revise_number",
        "prev_contract_id",
        "is_revised",
        "operator_name",
        "hotel_id",
        "hotel_name_gwg",
        "main_season",
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
        "meal_plan",
        "history",
        "activation_date",
    ],
    usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 20, 21, 22, 23, 24, 25, 26, 35, 37],
    dtype={
        "contract_id": pd.Int64Dtype(),
        "contract_type": str,
        "status": str,
        "purchase_contract_id": pd.Int64Dtype(),
        "revise_number": pd.Int64Dtype(),
        "prev_contract_id": pd.Int64Dtype(),
        "is_revised": bool,
        "operator_name": object,
        "hotel_id": pd.Int64Dtype(),
        "hotel_name_gwg": str,
        "main_season": str,
        "meal_plan": str,
        "history": str,
    },
    parse_dates=[
        "create_date",
        "begin_date",
        "end_date",
        "res_start_date",
        "res_end_date",
        "activation_date",
    ],
)

contracts.sort_values(by=["hotel_name_gwg", "create_date"], inplace=True)
contracts.drop_duplicates(subset="hotel_name_gwg", keep="last", inplace=True)

contracts["hotel_name"] = contracts["hotel_name_gwg"].str.split("(").str[0]
contracts["hotel_name"] = contracts["hotel_name"].str.split().str.join(" ")

In [5]:
contracts.to_csv("out.csv")

In [6]:
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

booking_data = BookingData("DU", "11/01/2022 00:00:00", "10/31/2024 00:00:00")
get_bookings = booking_data.get()

In [7]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
    ]
)

bookings = pipeline.fit_transform(get_bookings)

In [8]:
bookings.head()

,ref_id,res_id,hotel_id,operator_id,operator_code,bkg_ref,guest_name,sales_date,in_date,out_date,...,status5,purchase_contract_id,purchase_spo_id,sales_contract_id,sales_spo_id,sales_spo_name,sales_spo_code,purchase_spo_name,purchase_spo_code,main_season
0,1535416,778062,218736,6,AUH277,RDUMMY0208,RDUMMY0208 RDUMMY0208,2017-01-01,2023-03-09,2023-03-15,...,Man,291586,0,0,0,NaN,NaN,NaN,NaN,NaN
1,2110332,778062,218736,6,NaN,RDUMMY0208,test test,2017-01-01,2023-09-08,2023-09-13,...,Man,306694,0,0,0,NaN,NaN,NaN,NaN,NaN
2,2110333,778062,218736,6,NaN,RDUMMY0208,TEST TEST,2017-01-01,2023-09-08,2023-09-13,...,Man,306694,0,0,0,NaN,NaN,NaN,NaN,NaN
3,1721293,778062,218736,6,AUH277,RDUMMY0208,RDUMMY0208 RDUMMY0208,2017-01-01,2023-03-09,2023-03-15,...,Man,306694,0,0,0,NaN,NaN,NaN,NaN,NaN
4,2110330,778062,218736,6,NaN,RDUMMY0208,TEST TEST,2017-01-01,2023-09-08,2023-09-13,...,Man,306694,0,0,0,NaN,NaN,NaN,NaN,NaN


In [9]:
fti_group = [87, 63, 27, 88, 7, 915, 604, 6, 938, 509, 64, 19]
bookings = bookings[bookings["operator_id"].isin(fti_group)]

In [10]:
booking_counts = bookings["hotel_id"].value_counts().reset_index()
booking_counts.columns = ["hotel_id", "booking_count"]

In [11]:
merged_df = pd.merge(contracts, booking_counts, on="hotel_id", how="left")

In [12]:
merged_df["booking_count"] = merged_df["booking_count"].fillna(0)

In [13]:
merged_df.to_csv("out.csv")

In [1]:
from config import Config

Config.SSRS_USERNAME

'RS_AMS01_FTIAS_VIEW'

In [2]:
Config.SSRS_PASSWORD

'Ft!r3p0rt$AMS'

In [3]:
Config.SSRS_BASE_URL

'http://95.211.42.206/ReportServer_SSRS?/'